# Programming Assignment 4: Linear Classifers
Group 40: Albin Jansfelt, Amanda Allgurén, Lukas Martinsson

## Exercise Questions

**Why could the classifier "memorize" the training data in the first case, but not in the second case?**

The Perceptron and LinearSVC tries to linearly separate the groups in training. In the second case, all combinations have different output labels. Meaning *(Sydney, July) -> rain* and *(Sydney, December) -> sun*. In the first case *(Gothenburg,July) -> rain* and *(Gothenburg, December) -> rain*. This makes it linearly separable. The model has not memorized the training data, but found a linear function to separate it as best as possible.

The second case can be compared to an XOR gate. We have the four data points in four corners where the same label (sun/rain) is diagonal to eachother, meaning it is impossible to linearly separate them.

The change of city names from Gothenburg to Sydney does not affect the outcome of the classifiers.

#### Results:
Perceptron: Training time: 3.39 sec, Accuracy: 0.7919 \
Pegasos algorithm SVC: Training time: 5.33 sec, Accuracy: 0.8326 \
Pegasos algorithm Logistic: Training time: 4.40 sec, Accuracy: 0.8317 \
a) Faster linear algebra operations: Training time: 2.62 sec, Accuracy: 0.8326 \
b) Using sparse vectors: Training time: 4.09 sec, Accuracy: 0.8326 \
c) Speeding up the scaling operation: Training time: 3.89 sec, Accuracy: 0.8338

In [ ]:
import time
import numpy as np
import sys
import scipy.linalg.blas as blas
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from aml_perceptron import Perceptron, SparsePerceptron, LinearClassifier

### Original Perceptron

In [ ]:
# This function reads the corpus, returns a list of documents, and a list
# of their corresponding polarity labels. 
def read_data(corpus_file):
    X = []
    Y = []
    with open(corpus_file, encoding='utf-8') as f:
        for line in f:
            _, y, _, x = line.split(maxsplit=3)
            X.append(x.strip())
            Y.append(y)
    return X, Y

if __name__ == '__main__':
    
    # Read all the documents.
    X, Y = read_data('data/all_sentiment_shuffled.txt')
    
    # Split into training and test parts.
    Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.2,
                                                    random_state=0)

    # Set up the preprocessing steps and the classifier.
    pipeline = make_pipeline(
        TfidfVectorizer(),
        SelectKBest(k=1000),
        Normalizer(),

        # NB that this is our Perceptron, not sklearn.linear_model.Perceptron
        Perceptron()  
    )

    # Train the classifier.
    t0 = time.time()
    pipeline.fit(Xtrain, Ytrain)
    t1 = time.time()
    print('Training time: {:.2f} sec.'.format(t1-t0))

    # Evaluate on the test set.
    Yguess = pipeline.predict(Xtest)
    print('Accuracy: {:.4f}.'.format(accuracy_score(Ytest, Yguess)))



Training time: 3.39 sec.
Accuracy: 0.7919.


### Pegasos algorithm: SVC

In [ ]:
class Pegasos(LinearClassifier):
    """
    A straightforward implementation of the perceptron learning algorithm.
    """

    def __init__(self, n_iter=10, lam=1/10000):
        """
        The constructor can optionally take a parameter n_iter specifying how
        many times we want to iterate through the training set.
        """
        self.n_iter = n_iter
        self.lam = lam

    #lamda parameter added
    def fit(self, X, Y):
        """
        Train a linear classifier using the perceptron learning algorithm.
        """
        
        # First determine which output class will be associated with positive
        # and negative scores, respectively.
        self.find_classes(Y)

        # Convert all outputs to +1 (for the positive class) or -1 (negative).
        Ye = self.encode_outputs(Y)

        # If necessary, convert the sparse matrix returned by a vectorizer
        # into a normal NumPy matrix.
        if not isinstance(X, np.ndarray):
            X = X.toarray()

        # Initialize the weight vector to all zeros.
        n_features = X.shape[1]
        self.w = np.zeros(n_features)

        # Perceptron algorithm:
        '''
        for i in range(self.n_iter):
            for x, y in zip(X, Ye):

                # Compute the output score for this instance.
                score = x.dot(self.w)

                # If there was an error, update the weights.
                if y*score <= 0:
                    self.w += y*x
        '''

        #Pegasos algorithm, SVC
        t=0
        for i in range(self.n_iter):
            for x,y in zip(X, Ye):
                t+=1
                eta = 1 / (self.lam*t)
                score = np.dot(self.w, x)
                if(y*score < 1):
                    self.w = (1-eta*self.lam)*self.w + (eta*y)*x
                else:
                    self.w *= (1-eta*self.lam)

In [ ]:
# This function reads the corpus, returns a list of documents, and a list
# of their corresponding polarity labels. 
def read_data(corpus_file):
    X = []
    Y = []
    with open(corpus_file, encoding='utf-8') as f:
        for line in f:
            _, y, _, x = line.split(maxsplit=3)
            X.append(x.strip())
            Y.append(y)
    return X, Y


if __name__ == '__main__':
    
    # Read all the documents.
    X, Y = read_data('data/all_sentiment_shuffled.txt')
    
    # Split into training and test parts.
    Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.2,
                                                    random_state=0)

    # Set up the preprocessing steps and the classifier.
    pipeline = make_pipeline(
        TfidfVectorizer(),
        SelectKBest(k=1000),
        Normalizer(),
        Pegasos(10, 1/len(Xtrain))  
    )
    alg = 1
    # Train the classifier.
    t0 = time.time()
    pipeline.fit(Xtrain, Ytrain)
    t1 = time.time()
    print('Training time: {:.2f} sec.'.format(t1-t0))

    # Evaluate on the test set.
    Yguess = pipeline.predict(Xtest)
    print('Accuracy Pegasos algorithm SVC: {:.4f}.'.format(accuracy_score(Ytest, Yguess)))

Training time: 5.33 sec.
Accuracy Pegasos algorithm SVC: 0.8326.


### Pegasos algorithm: Logistic

In [ ]:
class Pegasos(LinearClassifier):
    """
    A straightforward implementation of the perceptron learning algorithm.
    """

    def __init__(self, n_iter=10, lam=1/10000):
        """
        The constructor can optionally take a parameter n_iter specifying how
        many times we want to iterate through the training set.
        """
        self.n_iter = n_iter
        self.lam = lam

    #lamda parameter added
    def fit(self, X, Y):
        """
        Train a linear classifier using the perceptron learning algorithm.
        """
        
        # First determine which output class will be associated with positive
        # and negative scores, respectively.
        self.find_classes(Y)

        # Convert all outputs to +1 (for the positive class) or -1 (negative).
        Ye = self.encode_outputs(Y)

        # If necessary, convert the sparse matrix returned by a vectorizer
        # into a normal NumPy matrix.
        if not isinstance(X, np.ndarray):
            X = X.toarray()

        # Initialize the weight vector to all zeros.
        n_features = X.shape[1]
        self.w = np.zeros(n_features)

        #Pegasos algorithm, Logistic
        t=0
        for i in range(self.n_iter):
            for x,y in zip(X, Ye):
                t += 1
                eta = 1 / (self.lam*t)
                score = np.dot(self.w, x)
                gradf = self.lam*self.w - np.divide(y, (1+np.exp(y*score)))*x
                self.w -= eta*gradf


In [ ]:
# This function reads the corpus, returns a list of documents, and a list
# of their corresponding polarity labels. 
def read_data(corpus_file):
    X = []
    Y = []
    with open(corpus_file, encoding='utf-8') as f:
        for line in f:
            _, y, _, x = line.split(maxsplit=3)
            X.append(x.strip())
            Y.append(y)
    return X, Y


if __name__ == '__main__':
    
    # Read all the documents.
    X, Y = read_data('data/all_sentiment_shuffled.txt')
    
    # Split into training and test parts.
    Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.2,
                                                    random_state=0)

    # Set up the preprocessing steps and the classifier.
    pipeline = make_pipeline(
        TfidfVectorizer(),
        SelectKBest(k=1000),
        Normalizer(),
        Pegasos(10, 1/len(Xtrain))  
    )
    alg = 1
    # Train the classifier.
    t0 = time.time()
    pipeline.fit(Xtrain, Ytrain)
    t1 = time.time()
    print('Training time: {:.2f} sec.'.format(t1-t0))

    # Evaluate on the test set.
    Yguess = pipeline.predict(Xtest)
    print('Accuracy Pegasos algorithm Logistic: {:.4f}.'.format(accuracy_score(Ytest, Yguess)))

Training time: 4.40 sec.
Accuracy Pegasos algorithm Logistic: 0.8317.


### Bonus task 1
#### a) Faster linear algebra operations

In [ ]:
class PegasosImproved(LinearClassifier):
    """
    A straightforward implementation of the perceptron learning algorithm.
    """

    def __init__(self, n_iter=10, lam=1/10000):
        """
        The constructor can optionally take a parameter n_iter specifying how
        many times we want to iterate through the training set.
        """
        self.n_iter = n_iter
        self.lam = lam

    #lamda parameter added
    def fit(self, X, Y):
        """
        Train a linear classifier using the perceptron learning algorithm.
        """
        
        # First determine which output class will be associated with positive
        # and negative scores, respectively.
        self.find_classes(Y)

        # Convert all outputs to +1 (for the positive class) or -1 (negative).
        Ye = self.encode_outputs(Y)

        # If necessary, convert the sparse matrix returned by a vectorizer
        # into a normal NumPy matrix.
        if not isinstance(X, np.ndarray):
            X = X.toarray()

        # Initialize the weight vector to all zeros.
        n_features = X.shape[1]
        self.w = np.zeros(n_features)

        #Pegasos algorithm improved, SVC
        t=0
        for i in range(self.n_iter):
            for x,y in zip(X, Ye):
                t+=1
                eta = 1 / (self.lam*t)
                score = blas.ddot(self.w, x)
            
                blas.dscal((1-eta*self.lam), self.w)
                if(y*score < 1):
                    blas.daxpy(x,self.w,a=eta*y)

In [ ]:

# Set up the preprocessing steps and the classifier.
pipeline = make_pipeline(
    TfidfVectorizer(),
    SelectKBest(k=1000),
    Normalizer(),
    #bestäm rimligt lambda
    PegasosImproved(10, 1/len(Xtrain))  
)


# Train the classifier.
t0 = time.time()
pipeline.fit(Xtrain, Ytrain)
t1 = time.time()
print('Training time: {:.2f} sec.'.format(t1-t0))

# Evaluate on the test set.
Yguess = pipeline.predict(Xtest)
print('Accuracy: {:.4f}.'.format(accuracy_score(Ytest, Yguess)))


Training time: 2.62 sec.
Accuracy: 0.8326.


#### b) Using sparse vectors

In [ ]:
def add_sparse_to_dense(x, w, factor):
    """
    Adds a sparse vector x, scaled by some factor, to a dense vector.
    This can be seen as the equivalent of w += factor * x when x is a dense
    vector.
    """
    w[x.indices] += factor * x.data
    return w

def sparse_dense_dot(x, w):
    """
    Computes the dot product between a sparse vector x and a dense vector w.
    """
    return np.dot(w[x.indices], x.data)

class SparseSVC(LinearClassifier):
    """
    A straightforward implementation of the perceptron learning algorithm.
    """

    def __init__(self, n_iter=10, lam=1/10000):
        """
        The constructor can optionally take a parameter n_iter specifying how
        many times we want to iterate through the training set.
        """
        self.n_iter = n_iter
        self.lam = lam

    #lamda parameter added
    def fit(self, X, Y):

        self.find_classes(Y)

        # First determine which output class will be associated with positive
        # and negative scores, respectively.
        Ye = self.encode_outputs(Y)

        # Initialize the weight vector to all zeros.
        #print(X.shape[1])
        self.w = np.zeros(X.shape[1])

        # Iteration through sparse matrices can be a bit slow, so we first
        # prepare this list to speed up iteration.
        XY = list(zip(X, Ye))
        
        #Pegasos algorithm, SVC 
   
        t=0
        for i in range(self.n_iter):
            for x,y in XY:
                t+=1
                eta = 1 / (self.lam*t)
                score = sparse_dense_dot(x, self.w)
                self.w *= (1-eta*self.lam)
             
                if(y*score < 1):
                    self.w = add_sparse_to_dense(x,self.w,(eta*y))
                    

In [ ]:
# Set up the preprocessing steps and the classifier.
pipeline = make_pipeline(
    TfidfVectorizer(),
    SelectKBest(k=1000),
    Normalizer(),
    #bestäm rimligt lambda
    SparseSVC(10, 1/len(Xtrain))  
)

# Train the classifier.
t0 = time.time()
pipeline.fit(Xtrain, Ytrain)
t1 = time.time()
print('Training time: {:.2f} sec.'.format(t1-t0))

# Evaluate on the test set.
Yguess = pipeline.predict(Xtest)
print('Accuracy: {:.4f}.'.format(accuracy_score(Ytest, Yguess)))


Training time: 4.09 sec.
Accuracy: 0.8326.


#### c) Speeding up the scaling operation

In [ ]:
def add_sparse_to_dense(x, w, factor):
    """
    Adds a sparse vector x, scaled by some factor, to a dense vector.
    This can be seen as the equivalent of w += factor * x when x is a dense
    vector.
    """
    w[x.indices] += factor * x.data
    return w

def sparse_dense_dot(x, w):
    """
    Computes the dot product between a sparse vector x and a dense vector w.
    """
    return np.dot(w[x.indices], x.data)

class SparseSVCImproved(LinearClassifier):
    """
    A straightforward implementation of the perceptron learning algorithm.
    """

    def __init__(self, n_iter=10, lam=1/10000):
        """
        The constructor can optionally take a parameter n_iter specifying how
        many times we want to iterate through the training set.
        """
        self.n_iter = n_iter
        self.lam = lam

    #lamda parameter added
    def fit(self, X, Y):

        self.find_classes(Y)

        # First determine which output class will be associated with positive
        # and negative scores, respectively.
        Ye = self.encode_outputs(Y)

        # Initialize the weight vector to all zeros.
        #print(X.shape[1])
        self.w = np.zeros(X.shape[1])


        # Iteration through sparse matrices can be a bit slow, so we first
        # prepare this list to speed up iteration.
        XY = list(zip(X, Ye))
        
        a=1
        t=1
        for i in range(self.n_iter):
            for x,y in XY:
                t+=1
                eta = 1 / (self.lam*t)
                score = a*sparse_dense_dot(x, self.w)
                a *= (1-eta*self.lam)
             
                if(y*score < 1):
                    add_sparse_to_dense(x,self.w,(eta*y)/a)
        self.w *= a           
                    

In [ ]:
pipeline = make_pipeline(
    TfidfVectorizer(),
    SelectKBest(k=1000),
    Normalizer(),
    #bestäm rimligt lambda
    SparseSVCImproved(10, 1/len(Xtrain))  
)

# Train the classifier.
t0 = time.time()
pipeline.fit(Xtrain, Ytrain)
t1 = time.time()
print('Training time: {:.2f} sec.'.format(t1-t0))

# Evaluate on the test set.
Yguess = pipeline.predict(Xtest)
print('Accuracy: {:.4f}.'.format(accuracy_score(Ytest, Yguess)))


Training time: 3.89 sec.
Accuracy: 0.8338.


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=7c023ec2-1210-4861-82d1-6d35a9cb94f6' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>